In [ ]:
#Reload all modules (except those excluded by %aimport) every time before executing the Python code typed.
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

data = pd.read_csv("/data/tm_goods_services_classification/tm_goods_services.csv",sep =',',header=None)
#df = pd.read_csv("/home/ubuntu/multilabel-text-classification/trademark_data_1003dsc.csv")
data.reset_index(drop=True, inplace=True)

data.head()

In [ ]:
#import sys
#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install tensorflow
#!{sys.executable} -m pip install keras
#!{sys.executable} -m pip install sklearn
#!{sys.executable} -m pip install nltk
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install google
#!{sys.executable} -m pip install future
#!{sys.executable} -m pip install np_utils
#!{sys.executable} -m pip install cython
#!{sys.executable} -m pip install fasttext
#!{sys.executable} -m pip  install h5py
#import nltk
#nltk.download('stopwords')

In [ ]:
import numpy as np
import pandas as pd
from __future__ import absolute_import
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
import pickle
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
import string #print (string.punctuation)
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import preprocessing

#Classifiers
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

#helpers
from pprint import pprint
from time import time
import logging,re

from sklearn.metrics import classification_report,confusion_matrix

import nltk
from nltk.corpus import stopwords
from string import punctuation
#initializersfrom keras import initializers
from keras.callbacks import CSVLogger
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, SeparableConv2D,BatchNormalization
from keras.layers import LSTM,Bidirectional,TimeDistributed,InputLayer,GRU
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Input, GlobalMaxPooling1D, Concatenate
from keras.layers import Conv1D, MaxPooling1D, Embedding, Conv2D, MaxPool2D, Reshape, Flatten
from keras.layers import SpatialDropout1D, Dense
from keras.optimizers import Adam
from keras.models import load_model
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.convolutional import Convolution2D,MaxPooling2D,Convolution1D,MaxPooling1D,UpSampling2D, ZeroPadding2D

In [ ]:
from workflow.data import get_trademark_data
from workflow.data import prepare_Xy
from workflow.data import prepare_Xy_goodsservices_data

from workflow.utilities import save_object
from workflow.utilities import load_object

#from workflow.fasttextwrap import fasttextClassifier

#from classifiers.kerasTextClassifier import create_bi_lstm_model
#from classifiers.kerasTextClassifier import create_model
#from workflow.train import getStopWords
#from workflow.train import trainFastTextModel
#from workflow.train import cleanData
#from workflow.train import clean_text

#from workflow.test import test_pred

In [ ]:
'''
##########################################
TODO
-add ability for stratified sampling
##########################################
'''
random_state = 47
test_size = .1
stopwords_file = 'stopwords/trademark_stopwords.txt'
data_dir = '/home/ubuntu/multilabel-text-classification'

In [ ]:
import os
def getStopWords():
    f = open(os.path.join(data_dir, stopwords_file))
    stopWordsList = f.read().splitlines()
    return stopWordsList

In [ ]:
stopwordsList = getStopWords()
en_stop = set(stopwords.words('english'))
en_stop.update(['.','(','(', ')','u',',', '"', "'", '?', '!', ':', ';' ,'(', ')', '[', ']', '{', '}', '&', '/', '-', '+', '*',\
                       "..", '...', '....', "+...", "-...", ",...", "'...", "!!!", "&...", "(...", ")...", "]...", "/...", "(+)...", "),...",\
                       "),", ").", "):", ")-", "))", "])", ".)", "!)", "')", "][", '").', '")/', '",', '"-', '")', '"/', './', '--',\
                       "#:", "(+)", "($", "-$", "/","+/",",", "+-", "(#", "''"])
stopWords = en_stop.union(stopwordsList)
#stopWords = en_stop
#print (stopWords)

In [ ]:
'''
##########################################
TODO
-Add ability to select subset of labels
-Add ability to set data_dir (various impacts throughout files)
##########################################

data = get_trademark_data(
    statement_url='https://bulkdata.uspto.gov/data/trademark/casefile/economics/2016/statement.csv.zip',
    design_url='https://bulkdata.uspto.gov/data/trademark/casefile/economics/2016/design_search.csv.zip',
    sample_size=1, #1 means 100% of the data. Lower it (.1) to run faster when testing script
    test_size=.2, #Percentage of data to set aside for testing
    filename='trademark_data.csv', #save retrieved data as this name, if exists will load .csv instead of url download
    data_dir='', #save retrieved data in this folder
    force_download=False #redownload from url and overwrite and saved .csv
    )

data.head()

X,y = prepare_Xy(data)
'''

In [ ]:
X,y = prepare_Xy_goodsservices_data(data)

In [ ]:
'''
##########################################
TODO
-add analysis jupyter workbook

##########################################

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)
'''

In [ ]:
import copy
copyofX = copy.deepcopy(X)
copyofy = copy.deepcopy(y)
#copyofX
#len(copyofX)
#len(copyofy)
#print (copyofy[0:2])

In [ ]:
def __remove_stop_words(text):
    return ' '.join([word for word in text.split() if word.lower() not in stopWords])

In [ ]:
shortword = re.compile(r'\W*\b\w{1,2}\b')
myList= []
for sentence in copyofX :
    #print (sentence)
    sentence= shortword.sub('', sentence)
    sentence = __remove_stop_words(sentence)
    #sentence=re.sub('([^\s\w]|_)+',' ',sentence).strip() 
    #sentence=re.sub('[?!\\,]+','',sentence).strip() #remove comma from sentence
    #sentence=re.sub('[?!\\.]+','',sentence).strip() #remove punctuation from sentence because we don't need it anymore
    #sentence=re.sub('[?!\\"]+','',sentence).strip() #remove quotes from sentence
    #sentence=re.sub('[?!\\-]+','',sentence).strip() #remove dash from sentence
    sentence=re.sub('\s+',' ',sentence)	# remove multiple spaces
    myList.append(sentence.lower())
    #print (sentence.lower())
print (len(myList))

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

# Transform name species into numerical values 
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)
y = np_utils.to_categorical(y)
#print(new_Y)
print (y.shape)

In [ ]:
#thefile = open('test.txt', 'w')
#for item in myList:
 # thefile.write("%s\n" % item)

In [ ]:
MAX_SEQUENCE_LENGTH = 200
MAX_NB_WORDS = 50000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1

# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(myList)
sequences = tokenizer.texts_to_sequences(myList)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
#labels = to_categorical(np.asarray(labels)) #aaron removed
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y.shape)


In [ ]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
y = y[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = y[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = y[-num_validation_samples:]

print('shape of:', x_train.shape)
print('shape of:',y_train.shape)
print('shape of:',x_val.shape)
print('shape of:',y_val.shape)

In [ ]:
#print('Preparing embedding matrix.')
#embeddings_index = {}
# prepare embedding matrix
#num_words = min(MAX_NB_WORDS, len(word_index))
#embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
#for word, i in word_index.items():
#    if i >= MAX_NB_WORDS:
#        continue
#    embedding_vector = embeddings_index.get(word)
#    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
#        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
#embedding_layer = Embedding(num_words,
#                            EMBEDDING_DIM,
#                            weights=[embedding_matrix],
#                            input_length=MAX_SEQUENCE_LENGTH,
#                            trainable=False)

In [ ]:
import os
from keras.layers import Embedding

BASE_DIR = ''
#GLOVE_DIR = os.path.join(BASE_DIR, 'glove_embedding')
#TEXT_DATA_DIR = os.path.join(BASE_DIR, 'glove.6B.300d.txt')


GLOVE_DIR = os.path.join(BASE_DIR, 'tm_goods_services_classification')
TEXT_DATA_DIR = os.path.join(BASE_DIR, 'tm_goods_services.bin')

MAX_SEQUENCE_LENGTH = 200
MAX_NB_WORDS = 50000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1

# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'tm_goods_services.vec'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

len(word_index)

In [ ]:
print('Training model.')

#sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
#embedded_sequences = embedding_layer(sequence_input)
#x = Conv1D(128, 3, activation='relu')(embedded_sequences)
#x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
#x = GlobalMaxPooling1D()(x)
#x = Dense(50, activation="relu")(x)
#x = Dropout(0.1)(x)
#preds = Dense(y_train.shape[1], activation='sigmoid')(x) # for multilabel classification # length of labels
#model = Model(sequence_input, preds)

#sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
#embedded_sequences = embedding_layer(sequence_input)
#x = Conv1D(128, 5, activation='relu')(embedded_sequences)
#x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
#x = MaxPooling1D(5)(x)
#x = Conv1D(128, 5, activation='relu')(x)
#x = MaxPooling1D(35)(x)
#x = Conv1D(128, 1, activation='relu')(x)
#x = GlobalMaxPooling1D()(x)
#x = Dense(128, activation='relu')(x)
#preds = Dense(y_train.shape[1], activation='sigmoid')(x) # for multilabel classification # length of labels
#model = Model(sequence_input, preds)

In [ ]:
def bigru_cnn():
    print('Loading model.')
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    units = 128
    conv_filters = 32
    x = Dropout(0.2)(embedded_sequences)
    x = Bidirectional(GRU(
    units,
    dropout=0.2,
    recurrent_dropout=0.2,
    return_sequences=True))(x)
    x = Reshape((2 * MAX_SEQUENCE_LENGTH, units, 1))(x)
    x = Conv2D(conv_filters, (3, 3))(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    preds = Dense(y_train.shape[1], activation='sigmoid')(x) # for multilabel classification # length of labels
    model = Model(sequence_input, preds)
    return model

In [ ]:
model = bigru_cnn()
model.load_weights('/data/tm_goods_services_classification/tm_goods_services.hdf5.old')

In [ ]:
'''
#mlb = load_object("test_mlb.sav")
prediction_prob=.3
#print (number.inverse_transform(new_X_test))
y_pred_results = model.predict(x_val, verbose=0)
#print (y_pred_results)
y_pred = np.where (y_pred_results > prediction_prob, 1, 0)
#print (y_pred)
'''

In [ ]:
#print (y_val)

In [ ]:
'''
print ((encoder.classes_))
x = map(str, encoder.classes_)
new_classes = list(x)
print(new_classes)
print (len (encoder.classes_))
'''

In [ ]:
#print(classification_report(y_val, y_pred)) #predict_proba scores by class

In [ ]:
#print (map(str, encoder.classes_))

In [ ]:
#mlb = load_object("test_mlb.sav")
'''
print(classification_report(y_val, y_pred, target_names=new_classes)) #predict_proba scores by class
print( classification_report( y_val, y_pred ,target_names=new_classes) , file=open('new_300d_goods_services_BI_GRU_tm_embedding_0.3.txt', 'w'))
'''

In [ ]:
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical
    
#test_txt = ['LEAD PENCILS, COLORED PENCILS AND COPYING INK PENCILS, INK AND PENCIL ERASERS']

#88042125
test_txt1 = ['Computer programming; Computer software consultancy; Computer software design; Creating and maintaining web sites for others; Industrial and graphic art design; Installation of computer software; Repair of computer software; Updating of computer software; Video game development services; Planning, design and implementation of computer technologies for others']

test_txt1 = __remove_stop_words(test_txt1[0])
test_txt1=re.sub('([^\s\w]|_)+',' ',test_txt1).strip() 
test_txt1=re.sub('\s+',' ',test_txt1)	# remove multiple spaces
test_txt1 = [str (test_txt1.lower())]
#print (test_txt1)
#test_txt = ['stylized lion circle', 'mermaid cup vapor']
prediction_prob=.7

MAX_SEQUENCE_LENGTH = 200
MAX_NB_WORDS = 50000
VALIDATION_SPLIT = 1
#print (len(myList))
#print (len(test_txt1))
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(myList)
prediction_sequences = tokenizer.texts_to_sequences(test_txt1)
#print (sequences)
prediction_word_index = tokenizer.word_index
#print('Found %s unique tokens.' % len(prediction_word_index))

prediction_data = pad_sequences(prediction_sequences, maxlen=MAX_SEQUENCE_LENGTH)
#print (prediction_data)
#labels = to_categorical(np.asarray(labels)) #aaron removed
#print('Shape of data tensor:', prediction_data.shape)
#print('Shape of label tensor:', y.shape)

indices = np.arange(prediction_data.shape[0])
#np.random.shuffle(indices)
prediction_data = prediction_data[indices]
prediction_y1 = y[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

prediction_x_train = prediction_data[:-num_validation_samples]
prediction_y_train = prediction_y1[:-num_validation_samples]
prediction_x_val = prediction_data[-num_validation_samples:]
prediction_y_val = prediction_y1[-num_validation_samples:]

#print('shape of x_train:', prediction_x_train.shape)
#print('shape of y_train:',prediction_y_train.shape)
#print('shape of x_val:',prediction_x_val.shape)
#print('shape of y_val:',prediction_y_val.shape)


test_prediction = model.predict(prediction_x_val, verbose=0)
#print (test_prediction)
new_test_prediction = np.where (test_prediction > prediction_prob, 1, 0)
#print (new_test_prediction)
new_predictions_list = [x for x in encoder.classes_[np.nonzero(new_test_prediction)[1]]]
print ("International class for serial no 88042125 : ", new_predictions_list)